In [1]:
import time
import os
from dotenv import load_dotenv
from binance.um_futures import UMFutures
from data_loader import DataLoader
key = os.getenv('BINANCE_API_KEY')
secret = os.getenv('BINANCE_SECRET_KEY')
client = UMFutures(key=key, secret=secret)
data_loader = DataLoader(client)

load_dotenv()


symbol = 'BTCUSDT'
interval = '1h' 


current_position = 0 


print(f"\n⏳ [{time.strftime('%H:%M:%S')}] --- New Loop ---")

# A. 抓取 Binance 數據
df = data_loader.get_binance_klines(symbol, interval, limit=200)

# B. 抓取 Google Trends 數據 (Data Loader 內部有做快取控制)
# 這裡傳入關鍵字 'Bitcoin'
google_data = data_loader.get_google_trends(['Bitcoin'])
print(f"   [Data] BTC價: {df['close'].iloc[-1]} | Google熱度: {google_data.get('Bitcoin', 'N/A')}")


⏳ [05:04:27] --- New Loop ---
🌍 正在向 Google 請求趨勢資料 (這可能需要幾秒鐘)...
   [Data] BTC價: 87601.2 | Google熱度: 77


d:\investment\量化策略實裝\.venv\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)


In [ ]:
import pandas as pd
bt_data=pd.read_csv('verify_result_Strategy1_MAD_BSR.csv')

In [13]:
log_keep=pd.read_csv(r'verify_result_Strategy9_Mom_VROC_Shock.csv')

In [14]:
import pandas as pd
import numpy as np

# 讀取資料
df = log_keep.copy()

# 確保時間排序
df = df.sort_values("timestamp").reset_index(drop=True)

# 1️⃣ 計算價格報酬（log return 或 simple return 擇一）
df["price_return"] = df["close_price"].astype(float).pct_change()

# 2️⃣ 建立持倉狀態（long=1, close=0）
df["position"] = (df["action"] == "LONG").astype(int)

# ⚠️ 關鍵：用「前一期持倉」吃當期報酬（避免 look-ahead bias）
df["strategy_return"] = df["position"].shift(1) * df["price_return"]

# 第一筆一定是 NaN
df = df.dropna()

# 3️⃣ Sharpe Ratio（以小時計）
mean_ret = df["strategy_return"].mean()
std_ret = df["strategy_return"].std()

sharpe_hourly = mean_ret / std_ret
annual_factor = np.sqrt(365 * 24)
sharpe_annual = sharpe_hourly * annual_factor


print("Annual Sharpe:", sharpe_annual)

Annual Sharpe: 1.7601680932575394


In [24]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('trading_data.db')
df = pd.read_sql("SELECT * FROM trades WHERE strategy='TEST_FORCE_BUY'", conn)
# 然後用 pandas 算 PnL

In [25]:
df

,id,timestamp,symbol,strategy,side,price,quantity,notional,order_id,fee
0,1,2026-01-06 19:11:13.161239,BTCUSDT,TEST_FORCE_BUY,BUY,0.00,0.002134,0.0000,11429948367,0.0
1,2,2026-01-06 21:25:17.657010,BTCUSDT,TEST_FORCE_BUY,BUY,0.00,0.002132,0.0000,11432413215,0.0
2,3,2026-01-06 21:28:41.721714,BTCUSDT,TEST_FORCE_BUY,BUY,0.00,0.002134,0.0000,11432476163,0.0
3,4,2026-01-06 23:11:39.367426,BTCUSDT,TEST_FORCE_BUY,LONG,93691.20,0.002000,187.3824,11434408178,0.0
4,5,2026-01-06 23:42:54.822230,BTCUSDT,TEST_FORCE_BUY,LONG,93316.30,0.002000,186.6326,11435005590,0.0
5,6,2026-01-06 23:53:08.785129,BTCUSDT,TEST_FORCE_BUY,LONG,93376.00,0.002000,186.7520,11435196678,0.0
6,7,2026-01-06 23:54:07.399939,BTCUSDT,TEST_FORCE_BUY,LONG,93390.16,0.005000,466.9508,11435216507,0.0
7,8,2026-01-07 00:58:06.309459,BTCUSDT,TEST_FORCE_BUY,LONG,92610.90,0.005000,463.0545,11436430350,0.0


In [28]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('trading_data.db')
df = pd.read_sql("SELECT * FROM trades WHERE strategy='TEST_LOOP'", conn)
# 然後用 pandas 算 PnL

In [29]:
df

,id,timestamp,symbol,strategy,side,price,quantity,notional,order_id,fee
0,10,2026-01-07 02:15:58.157403,BTCUSDT,TEST_LOOP,LONG,91846.50,0.005,459.2325,11437880816,0.0
1,11,2026-01-07 02:16:12.768525,BTCUSDT,TEST_LOOP,CLOSE,91797.20,0.005,458.9860,11437887651,0.0
2,12,2026-01-07 02:16:27.357009,BTCUSDT,TEST_LOOP,LONG,91799.90,0.005,458.9995,11437893978,0.0
3,13,2026-01-07 02:19:28.757090,BTCUSDT,TEST_LOOP,LONG,91786.72,0.005,458.9336,11437949059,0.0
4,14,2026-01-07 02:25:04.014003,BTCUSDT,TEST_LOOP,LONG,91822.30,0.005,459.1115,11438046009,0.0
5,15,2026-01-07 02:25:18.819050,BTCUSDT,TEST_LOOP,CLOSE,91824.02,0.005,459.1201,11438053059,0.0
6,16,2026-01-07 02:25:33.757857,BTCUSDT,TEST_LOOP,LONG,91866.50,0.005,459.3325,11438059543,0.0
7,17,2026-01-07 02:25:48.593897,BTCUSDT,TEST_LOOP,CLOSE,91863.30,0.005,459.3165,11438061404,0.0
8,18,2026-01-07 02:26:03.414021,BTCUSDT,TEST_LOOP,LONG,91872.80,0.005,459.3640,11438065472,0.0
9,19,2026-01-07 02:26:18.347242,BTCUSDT,TEST_LOOP,CLOSE,91868.40,0.005,459.3420,11438071562,0.0


In [48]:
import numpy as np

a = np.array([1, 2])
b = a
a = np.append(a, 3)

In [50]:
b

array([1, 2])

In [57]:
import numpy as np

x = [1]
a = np.array([x], dtype=object)
x.append(2)

# 問：a 是什麼？

In [53]:
import copy
import numpy as np

x = [[1]]
a = []

a.append(x)
a.append(x.copy())
a.append(copy.deepcopy(x))
a.append(np.array(x))

x[0].append(2)

# 問：a 是什麼？


In [73]:
def extend_list(val, list=[]):
    list.append(val)
    return list

list1 = extend_list(10)
list2 = extend_list(123, [])
list3 = extend_list('a')

print(f"list1 = {list1}")
print(f"list2 = {list2}")
print(f"list3 = {list3}")

list1 = [10, 'a']
list2 = [123]
list3 = [10, 'a']


In [76]:
funcs = [lambda x: x * i for i in range(4)]
print([f(10) for f in funcs])

[30, 30, 30, 30]


In [74]:
a = [1, 2, 3]
b = a
c = a[:]

a[0] = 99

print(b[0], c[0])

99 1


In [81]:
print([i for i in range(4)])

[0, 1, 2, 3]


In [82]:
d = {"a": 1, "b": 2}

In [85]:
import pandas as pd
s = pd.Series([10, 20, 30, 40], index=['a', 'b', 'c', 'd'])

In [ ]:
funcs = {}
for i in range(3):
    funcs[i] = lambda x: x * i

print( for k in funcs])


{(1, 2): 'A'}


In [94]:
funcs = {}
for i in range(3):
    funcs[i] = lambda x, i=i: x * i

print( [funcs[k](10) for k in funcs])


[0, 10, 20]


In [95]:
d = {"a": 1, "b": 2, "c": 3}

new_d = {
    k: v*2
    for k, v in map(lambda kv: (kv[0], kv[1]+1), d.items())
}

print(new_d)


{'a': 4, 'b': 6, 'c': 8}


In [97]:
ticks = [
    ("AAPL", 100),
    ("AAPL", 101),
    ("GOOG", 200),
    ("AAPL", 99),
]

last = {}

for sym, px in map(lambda x: x, ticks):
    last[sym] = px

print(last)


{'AAPL': 99, 'GOOG': 200}


In [109]:
k=np.array([3,2])+np.array([2])

In [ ]:
class Order:
    def __init__(self, id):
        self.id = id

    def __eq__(self, other):
        return self.id == other.id

o1 = Order(1)
o2 = Order(1)

d = {o1: "buy"}
print(d[o2])

In [ ]:
o1.

In [ ]:
class P:
    def __init__(self):
        self.x = 1

    @property
    def x(self):
        return self.x + 1

p = P()
print(p.x)


In [140]:
a = [1, 2, 3]
b = a[:]
b[0] = 10
c = a
c[0] = 20
print(a, b, c)

[20, 2, 3] [10, 2, 3] [20, 2, 3]


In [ ]:
# 用一行代碼實現：將股票列表按價格排序
stocks = [('AAPL', 150), ('GOOGL', 2800), ('TSLA', 700)]
sorted_stocks = sorted(stocks, key=lambda x: x[1])

GOOGL 2800


In [161]:
class F:
    def __new__(cls):
        return 1
    
a=F()

In [173]:
class A:
    @property
    def x(self):
        return 1

a = A()
a.__dict__['x'] = 999
print(a.x)

1


In [196]:
s = pd.Series([1,2,3])
df = pd.DataFrame(s)

In [209]:
s1 = pd.Series([1,2,3,4], index=["a","b","c","c"])
s1

a    1
b    2
c    3
c    4
dtype: int64

In [226]:
df = pd.DataFrame({
    "ret": [0.1, -0.2, 0.3],
    "signal": [0, 0, 0]
})

In [228]:
price = pd.Series([100, 110, 121])
ret = price.pct_change()

In [230]:
a = np.arange(6)
b = a[1:5]
b[0] = 99


In [ ]:
s1 = pd.Series([1, 2, 3], index=["a", "b", "c"])
s2 = pd.Series([10, 20, 30], index=["b", "c", "d"])

s3 = s1 + s2


array([ 0, 99,  2,  3,  4,  5])

In [252]:
s1 = pd.Series([1, 2, 3], index=["a", "b", "c"])
s2 = pd.Series([10, 20, 30], index=["b", "c", "d"])

s3 = s1.add(s2,fill_value=0)

In [253]:
s3

a     1.0
b    12.0
c    23.0
d    30.0
dtype: float64

In [232]:
df = pd.DataFrame(
    {"price": [100, 200, 300]},
    index=[10, 20, 30]
)

x = df.loc[20, "price"]
y = df.iloc[1, 0]


In [257]:
df = pd.DataFrame({
    "ret": [0.1, -0.2, 0.3],
    "signal": [0, 0, 0]
})

df[df["ret"] > 0]["signal"] = 1


C:\Users\User\AppData\Local\Temp\ipykernel_34080\356671784.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["ret"] > 0]["signal"] = 1


In [258]:
df

,ret,signal
0,0.1,0
1,-0.2,0
2,0.3,0


In [259]:
price = pd.Series([100, 110, 121])
ret = price.pct_change()
signal = ret > 0


In [268]:
df = pd.DataFrame(
    {"x": [100, 200, 300]},
    index=[10, 20, 30]
)

print(df.loc[1:2]["x"])


Series([], Name: x, dtype: int64)


In [305]:
df = pd.DataFrame({"x": [10, 20]}, index=["a", "b"])
print(a:=df.values)

[[10]
 [20]]


In [273]:
df = pd.DataFrame({"ret": [0.1, -0.2, 0.3], "sig": 0})
df[df["ret"] > 0]["sig"] = 1
print(df["sig"].values)

[0 0 0]


C:\Users\User\AppData\Local\Temp\ipykernel_34080\1722243898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["ret"] > 0]["sig"] = 1


In [294]:
a = np.arange(3).reshape(3, 1)
b = np.arange(6).reshape(3, 2)

print(a + b)

[[0 1]
 [3 4]
 [6 7]]


In [296]:
np.array([1, 2, 3], dtype=np.int32) * 1e9

array([1.e+09, 2.e+09, 3.e+09])

In [297]:
prices = np.array([100, 110, 120])    
a=prices[:, np.newaxis]


In [311]:
nums = [1,2,3]
nums.insert(0,10)

In [312]:
nums

[10, 1, 2, 3]

In [330]:
import pandas as pd

df = pd.DataFrame({
    '部門': ['業務', '業務', '研發', '研發', '研發'],
    '業績': [100, 200, 150, 300, 250],
    '獎金': [10, 20, 15, 30, 25]
})

# 同時計算業績的總和與平均，以及獎金的最大值
res_agg = df.groupby('部門')['業績'].transform('sum')

In [320]:
df

,部門,業績,獎金
0,業務,100,10
1,業務,200,20
2,研發,150,15
3,研發,300,30
4,研發,250,25


In [331]:
res_agg

0    300
1    300
2    700
3    700
4    700
Name: 業績, dtype: int64

In [13]:
from utils.config_loader import ConfigLoader
config = ConfigLoader("config.json")

In [17]:
strategy_names = config.get('trading', {})

In [21]:
strategy_names = config.get('trading','strategies', [])

In [22]:
strategy_names

['TestStrategy', 'PriceVolume2', 'TestStrategy2']